In [2]:
%cd ..

/home/ead/iking5/code/bitcoin_fraud_detection


In [13]:
import torch 
from models.inductive_graph_gan import Generator 
from inductive_graphgan import prepare_btc_data

g = torch.load('resources/btc_graph_unsplit.pt')
_,gsd, hidden, latent = torch.load('saved_models/inductive_gan.pt')
gen = Generator(g.x.size(1)-1, hidden, latent)
gen.load_state_dict(gsd)
gen.eval()

Generator(
  (first): Sequential(
    (0): Linear(in_features=165, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (mu): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=64, bias=True)
  )
  (logvar): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=64, bias=True)
  )
  (decode): Sequential(
    (0): Linear(in_features=64, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=1024, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=1024, out_features=165, bias=True)
  )
)

In [14]:
x = g.x[:,1:]
y = g.y 

with torch.no_grad():
    z = gen.encode(x)

In [15]:
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    f1_score, accuracy_score, precision_score, recall_score
)

kfold = StratifiedKFold()
preds = []
ys = []
i = 0 
for tr, te in kfold.split(z, y):
    print(i)
    rf = RF(n_jobs=16)
    rf.fit(z[tr], y[tr])
    pred = rf.predict(z[te])

    preds.append(pred)
    ys.append(y[te])

    i += 1

0
1
2
3
4


In [16]:
import numpy as np 
all_preds = torch.from_numpy(np.concatenate(preds))
all_ys = torch.cat(ys)

# Remove unk 
is_unk = all_ys == 0  
all_preds = all_preds[~is_unk]
all_ys = all_ys[~is_unk]

# Reindex s.t. mal is 1, benign is 0 
all_preds[all_preds == 2] = 0 
all_ys[all_ys == 2] = 0 

# Get stats
print("Accuracy: ", accuracy_score(all_ys, all_preds))
print("Precision: ", precision_score(all_ys, all_preds))
print("Recall: ", recall_score(all_ys, all_preds))
print("F1: ", f1_score(all_ys, all_preds))

Accuracy:  0.9131732669014689
Precision:  0.9114754098360656
Recall:  0.12233223322332233
F1:  0.2157129000969932


In [18]:
tr_z = z[g.x[:,0] < 35]
tr_y = y[g.x[:,0] < 35]

is_unk = tr_y == 0 
tr_y = tr_y[~is_unk]
tr_z = tr_z[~is_unk]
tr_y[tr_y == 2] = 0

te_z = z[g.x[:,0] >= 35]
te_y = y[g.x[:,0] >= 35]

is_unk = te_y == 0 
te_y = te_y[~is_unk]
te_z = te_z[~is_unk]
te_y[te_y == 2] = 0 

rf = RF(n_jobs=16)
rf.fit(tr_z, tr_y)
preds = rf.predict(te_z)

print("Accuracy: ", accuracy_score(te_y, preds))
print("Precision: ", precision_score(te_y, preds))
print("Recall: ", recall_score(te_y, preds))
print("F1: ", f1_score(te_y, preds))

Accuracy:  0.9307738452309539
Precision:  0.44301765650080255
Recall:  0.2548476454293629
F1:  0.3235638921453693
